In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore", FutureWarning)

In [2]:
dataset = "News.json"

if os.path.isfile(dataset):
    news = pd.read_json("News.json", lines=True)

display(news.head(10))

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."
5,Ron Dicker,ENTERTAINMENT,2018-05-26,Morgan Freeman 'Devastated' That Sexual Harass...,https://www.huffingtonpost.com/entry/morgan-fr...,"""It is not right to equate horrific incidents ..."
6,Ron Dicker,ENTERTAINMENT,2018-05-26,Donald Trump Is Lovin' New McDonald's Jingle I...,https://www.huffingtonpost.com/entry/donald-tr...,"It's catchy, all right."
7,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Amazon Prime That’s New This ...,https://www.huffingtonpost.com/entry/amazon-pr...,There's a great mini-series joining this week.
8,Andy McDonald,ENTERTAINMENT,2018-05-26,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,https://www.huffingtonpost.com/entry/mike-myer...,"Myer's kids may be pushing for a new ""Powers"" ..."
9,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Hulu That’s New This Week,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...


In [3]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
authors              200853 non-null object
category             200853 non-null object
date                 200853 non-null datetime64[ns]
headline             200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [4]:
news.drop(columns=['date', 'link', 'authors', 'short_description'], axis=1, inplace=True)
news.head()

,category,headline
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [5]:
# as shown above, THE WORLDPOST and WORLDPOST should be the same category, so merge them. (and ohers as well)
def category_cleaner(x):
    
    if x == 'THE WORLDPOST':
        return 'WORLDPOST'
    elif x == 'PARENTING':
        return 'PARENTS'
    elif x == 'ARTS' or x == 'CULTURE & ARTS':
        return 'ARTS & CULTURE'
    elif x == 'STYLE':
        return 'STYLE & BEAUTY'
    elif x == 'COLLEGE':
        return 'EDUCATION'
    elif x == 'TASTE':
        return 'FOOD & DRINK'
    else:
        return x
    
news['category'] = news.category.apply(category_cleaner)


In [6]:
cats = news.groupby('category')
print("total categories:", cats.ngroups)
print(cats.size().sort_values(ascending=False))
#print(type(cats.size()))

total categories: 34
category
POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
PARENTS           12632
STYLE & BEAUTY    11903
TRAVEL             9887
FOOD & DRINK       8322
HEALTHY LIVING     6694
QUEER VOICES       6314
WORLDPOST          6243
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
ARTS & CULTURE     3878
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
RELIGION           2556
SCIENCE            2178
WORLD NEWS         2177
EDUCATION          2148
TECH               2082
MONEY              1707
FIFTY              1401
GOOD NEWS          1398
ENVIRONMENT        1323
LATINO VOICES      1129
dtype: int64


In [7]:
news.head()

,category,headline
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [8]:
from nltk.tokenize import TweetTokenizer
politics = news[news.category == 'POLITICS']

# Use TweetTokenizer 
tknzr = TweetTokenizer()
politics['text'] = politics['headline'].apply(tknzr.tokenize)

politics.reset_index(inplace=True)

C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
contains_Donald = []

for i in range(len(politics.text)):
    if 'Donald' in politics.text[i]:
        contains_Donald.append(1)
    else:
        contains_Donald.append(0)
        
politics['Donald'] = contains_Donald
politics[politics['Donald'] == 1].count()

C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


index       2969
category    2969
headline    2969
text        2969
Donald      2969
dtype: int64

In [10]:
from nltk.tokenize import TweetTokenizer
media = news[news.category == 'MEDIA']

# Use TweetTokenizer 
tknzr = TweetTokenizer()
media['text'] = media['headline'].apply(tknzr.tokenize)

media.reset_index(inplace=True)



C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
contains_Donald2 = []

for i in range(len(media.text)):
    if 'Donald' in media.text[i]:
        contains_Donald2.append(1)
    else:
        contains_Donald2.append(0)
media['Donald'] = contains_Donald2     
media[media['Donald']==1].count()

C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


index       194
category    194
headline    194
text        194
Donald      194
dtype: int64

Now, we are prepare to do some hypothesis and test them. First, we state our hypothesis:

$H_0$: $\mu_{spam=call} = \mu_{ham=call}$  
$H_A$: $\mu_{spam=call} \neq \mu_{ham=call}$ 

Having $\alpha=0.05$  

In [12]:
print("Total word 'Donald':", 2969+194)
print("Total word 'Donald' in POLITICS: 2969")
print("Total word 'call' in MEDIA: 194")

Total word 'Donald': 3163
Total word 'Donald' in POLITICS: 2969
Total word 'call' in MEDIA: 194


In [16]:
no_Donald = [(32739-2969)/32739, (2815-194)/2815]
Donald = [2969/32739, 194/2815]
print(no_Donald)
print(Donald)

[0.9093130517120254, 0.9310834813499111]
[0.09068694828797459, 0.06891651865008881]


In [17]:
rates = pd.DataFrame(data=[no_Donald, Donald], index=["no 'Donald'", "'Donald'"])
rates.columns = ['POLITICS', 'MEDIA']
rates

,POLITICS,MEDIA
no 'Donald',0.909313,0.931083
'Donald',0.090687,0.068917


In [18]:
p_s = 2969/32739
p_h = 194/2815
sigma_s = (p_s * (1 - p_s)) / 32739
sigma_h = (p_h * (1 - p_h)) /2815


# difference sample
mu = p_s - p_h
sigma = sigma_s + sigma_h
std_dev = np.sqrt(sigma)
print(mu)

0.02177042963788578



Distance from $\mu$ is $d = z * std\_dev$ and according z-table $z=1.96$. Margin error is $2*d$. Keep in mind, our data is large enough, so we can use our sample proportions $\approx$ proportions.

In [19]:
d = 1.96*std_dev
low = mu - d
up = mu + d
margin_error = 2*d
margin_error
print("Low:",  low)
print("Up:", up)

Low: 0.01190917941774463
Up: 0.03163167985802693


Our $95\%$ confidence interval is $<0.011909, 0.0316317>$. And therfore we can reject null hypothesis. This is a result we would expect, to see word 'Donald' more often in 'Politics' then in 'Media' category.